####  Setup stuff: Connectivity

In [ ]:
import os

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#### Create the vertices/nodes, edges ..


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
import pandas as pd

print("--")


In [ ]:

df_airports = pd.DataFrame([
   ["MKE", "Milwaukee"                     ],
   ["ORD", "Chicago O-Hare"                ],
   ["SJC", "San Jose"                      ],
   ["DEN", "Denver"                        ] ,
   ], columns = ['airport_code', 'airport_name'])

df_airports.head(10)


In [ ]:
#  Create nodes, airports


import contextlib

for l_each in df_airports.iterrows():
    
   l_airport_code = l_each[1][0]
   l_airport_name = l_each[1][1]
    
   l_query = """

      //  Was using,
      //
      // CREATE ( n: Airport {{ airport_code: '{0}' }} ) 
      // SET n.airport_name = '{1}' 
      
      //  Was told to use this, because it's better ..
      //
      MERGE ( n: Airport {{ airport_code: '{0}' }})
      ON CREATE SET n.airport_name = '{1}'
   
      """.format(l_airport_code, l_airport_name)
    
   l_result0 = my_graph.query(l_query)
        
   print(".", end='')


print("  ")
print("--")


In [ ]:
#  What's in the vertices/nodes ..

l_result = my_graph.query("""

   MATCH ( n ) 
   RETURN n
   
   """)

print(l_result[0:10])


#  When using CREATE code above
#
#     n.internal_id   n.labels n.airport_code  n.airport_name n.type
#  0              0  [Airport]            MKE       Milwaukee   node
#  1              1  [Airport]            ORD  Chicago O-Hare   node
#  2              2  [Airport]            SJC        San Jose   node
#  3              3  [Airport]            DEN          Denver   node

#  When using MERGE code above
#
#     n.internal_id   n.labels n.airport_code n.type n.airport_name
#  0              0  [Airport]            SJC   node            NaN
#  1              1  [Airport]            MKE   node      Milwaukee
#  2              2  [Airport]            ORD   node       San Jose
#  3              3  [Airport]            DEN   node         Denver

